## 서울시 구별 주유소 가격 정보 얻기

In [ ]:
from selenium import webdriver

In [ ]:
driver = webdriver.Chrome('chromedriver')
driver.get("http://www.opinet.co.kr/searRgSelect.do")

- 상단메뉴에서 [주유소/충전소 찾기]-[지역별] 에서 서울 입력 

### http://www.opinet.co.kr/searRgSelect.do#

### xpath 이용하기 

- html에서 [Copy]-[Copy xPAth]
- 변수 = driver.find_element_by_xpath("""xPath""")
- 변수.find_elements_by_tag_name("태그명")

In [ ]:
# 구 옵션메뉴 클릭 
# [F12] 소스 편집창에서 [Copy]-[Xpath]
# //*[@id="SIGUNGU_NM0"]

gu_list_raw = driver.find_element_by_xpath("""//*[@id="SIGUNGU_NM0"]""")
gu_list = gu_list_raw.find_elements_by_tag_name("option")

In [ ]:
# gu_list

### "value" 값으로 구이름 추출하기 

In [ ]:
gu_names = [option.get_attribute("value") for option in gu_list]
# gu_names
gu_names.remove('')
# gu_names

In [ ]:
# 강남구 지정하기 
#  #SIGUNGU_NM0 는 2번째 옵션 
element = driver.find_element_by_id("SIGUNGU_NM0")
element.send_keys(gu_names[0])

In [ ]:
# [조회] a 태그의 xpath 조회한 후 클릭 
# //*[@id="searRgSelect"]

xpath = """//*[@id="searRgSelect"]"""
element_sel_gu = driver.find_element_by_xpath(xpath).click()

In [ ]:
# 강남구 주유소 검색 페이지가 표시 
# 하단 [엑셀저장] 의 xpath 조회 
# //*[@id="glopopd_excel"]

xpath = """//*[@id="glopopd_excel"]"""
element_get_excel = driver.find_element_by_xpath(xpath).click()

## 서울 모든 구에 대한 주유소 엑셀 파일 저장하기 

In [ ]:
import time
from tqdm import tqdm_notebook

for gu in tqdm_notebook(gu_names):
    element = driver.find_element_by_id("SIGUNGU_NM0")
    element.send_keys(gu)
    
    time.sleep(2)
    
    xpath = """//*[@id="searRgSelect"]"""
    element_sel_gu = driver.find_element_by_xpath(xpath).click()
    
    time.sleep(1)
    
    xpath = """//*[@id="glopopd_excel"]"""
    element_get_excel = driver.find_element_by_xpath(xpath).click()
    
    time.sleep(1)

In [ ]:
# 엑셀 파일이 다운로드 된다. 
# output_excel 폴더 생성후 다운로드한 파일 이동시키기 

In [ ]:
import os
os.listdir('output_excel')
len(os.listdir('output_excel'))

In [ ]:
driver.close()

## 구별 주유 가격에 대한 데이터의 정리

### golb 모듈의 glob() 를 이용하여 특정 폴더안의 엑셀 파일 리스트 생성

- 리스트변수 = glob.glob('폴더경로l/*.확장자')

In [ ]:
import pandas as pd
import glob

In [ ]:
glob.glob('output_excel/*.xls')

In [ ]:
stations_files = glob.glob('output_excel/*.xls')
stations_files

### 엑셀파일 모두 합치고 데이타프레임 생성 

In [ ]:
tmp_raw = []

for file_name in stations_files:
    tmp = pd.read_excel(file_name, header=2)
    tmp_raw.append(tmp)
    
station_raw = pd.concat(tmp_raw)

In [ ]:
station_raw.info()

In [ ]:
station_raw.head()

In [ ]:
station_raw.shape

### 필요한 컬럼명만 추출하기 

In [ ]:
stations = pd.DataFrame({'Oil_store':station_raw['상호'], 
                                       '주소':station_raw['주소'],
                                       '가격':station_raw['휘발유'],
                                       '셀프':station_raw['셀프여부'],
                                       '상표':station_raw['상표']  })
stations.head()

### 구별로 정리하기 

In [ ]:
stations['구'] = [eachAddress.split()[1] for eachAddress in stations['주소']]
stations.head()

In [ ]:
stations['구'].unique()

### 가격이 없는 경우 '-' 로 표시됨 

In [ ]:
stations[stations['가격']=='-']

In [ ]:
len(stations[stations['가격']=='-'])

### 가격이 '-' 인 경우 데이타프레임에서 삭제 

In [ ]:
stations = stations[stations['가격'] != '-']
stations.head()

In [ ]:
len(stations[stations['가격']=='-'])

### '가격' 컬럼을 숫자형으로 변경하기 

In [ ]:
stations['가격'].dtype

In [ ]:
# stations['가격'] = [float(value) for value in stations['가격']]
stations['가격'] = stations['가격'].astype(float)

In [ ]:
stations['가격'].dtype

### 인덱스 재정렬후 index 컬럼 삭제 

In [ ]:
stations.reset_index(inplace=True)
del stations['index']

In [ ]:
stations.info()

In [ ]:
stations.head()

In [ ]:
stations.describe()

### 서울에서 가장 가격이 저렴한 주유소 정보 출력하기 

In [ ]:
stations[stations['가격']==stations['가격'].min()]

##  셀프 주유소  boxplot으로 확인하기

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#  한글글꼴로 변경
plt.rcParams['font.size'] = 20.0
# plt.rcParams['font.family'] = 'batang'
plt.rcParams['font.family'] = 'Malgun Gothic'
print(plt.rcParams['font.family'])
print(plt.rcParams['font.size'])

In [ ]:
stations.boxplot(column='가격', by='셀프', figsize=(12,8));

In [ ]:
plt.figure(figsize=(12,8))
sns.boxplot(x="상표", y="가격", hue="셀프", data=stations, palette="Set3")
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
sns.boxplot(x="상표", y="가격", data=stations, palette="Set3")
sns.swarmplot(x="상표", y="가격", data=stations, color=".6")
plt.show()

## 서울시 구별 주유 가격 확인하기

In [ ]:
import json
import folium
import googlemaps
import warnings
warnings.simplefilter(action = "ignore", category = FutureWarning)

### 가격 값으로 역순 정렬

In [ ]:
stations.sort_values(by='가격', ascending=False).head(10)

In [ ]:
stations.sort_values(by='가격', ascending=True).head(10)

### 피봇 테이블로 구별로 가격 정리하기 

In [ ]:
import numpy as np

gu_data = pd.pivot_table(stations, index=["구"], values=["가격"], 
                         aggfunc=np.mean)
gu_data.head()

### map.choropleth()를 이용한 지도 시각화 

In [ ]:
geo_path = 'data/02. skorea_municipalities_geo_simple.json'
geo_str = json.load(open(geo_path, encoding='utf-8'))

map = folium.Map(location=[37.5502, 126.982], zoom_start=10.5, 
                 tiles='Stamen Toner')

map.choropleth(geo_data = geo_str,
               data = gu_data,
               columns=[gu_data.index, '가격'],
               fill_color='PuRd', #PuRd, YlGnBu
               key_on='feature.id')
map